In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Positions - Copy.csv")
df.head()

,Unnamed: 0,index,strategy,trade_type,notional,ticker,description,daily_pl,price_local
0,0,2189,EXAM:STRAT1,RatesBond,12513254.0,T 4 1/2 02/15/44,T 4 1/2 02/15/44,5457.0,95.984375
1,1,2190,EXAM:STRAT1,RatesBond,441178621.0,T 3 7/8 02/15/43,T 3 7/8 02/15/43,253769.0,88.390625
2,2,2191,EXAM:STRAT1,RatesBond,120984500.0,T 4 11/15/42,T 4 11/15/42,79458.0,90.101562
3,3,2192,EXAM:STRAT1,RatesBond,241969000.0,T 4 1/2 08/15/39,T 4 1/2 08/15/39,218952.0,98.203125
4,4,2217,EXAM:STRAT2,RatesBond,-206364990.0,T 4 5/8 02/28/26,T 4 5/8 02/28/26,-231495.0,99.480469


In [3]:
print(df.dtypes)

Unnamed: 0       int64
index            int64
strategy        object
trade_type      object
notional       float64
ticker          object
description     object
daily_pl       float64
price_local    float64
dtype: object


Making sure columns with numbers are in the correct format to avoid issues down the line

In [4]:
df['notional'] = pd.to_numeric(df['notional'], errors='coerce').fillna(0)
df['daily_pl'] = pd.to_numeric(df['daily_pl'], errors='coerce').fillna(0)
df['price_local'] = pd.to_numeric(df['price_local'], errors='coerce').fillna(0)

Clean data/remove unnecessary columns 

In [5]:
df_cleaned = df[(df['notional'] != 0)]

df_cleaned = df_cleaned.drop(columns=['Unnamed: 0', 'index'])

df_cleaned.head()

,strategy,trade_type,notional,ticker,description,daily_pl,price_local
0,EXAM:STRAT1,RatesBond,12513254.0,T 4 1/2 02/15/44,T 4 1/2 02/15/44,5457.0,95.984375
1,EXAM:STRAT1,RatesBond,441178621.0,T 3 7/8 02/15/43,T 3 7/8 02/15/43,253769.0,88.390625
2,EXAM:STRAT1,RatesBond,120984500.0,T 4 11/15/42,T 4 11/15/42,79458.0,90.101562
3,EXAM:STRAT1,RatesBond,241969000.0,T 4 1/2 08/15/39,T 4 1/2 08/15/39,218952.0,98.203125
4,EXAM:STRAT2,RatesBond,-206364990.0,T 4 5/8 02/28/26,T 4 5/8 02/28/26,-231495.0,99.480469


Double checking there are no zeros in the notional column before we move forward

In [6]:
print((df_cleaned['notional'] == 0).sum())

0


Getting a sense of how many strategies there are

In [7]:
unique_strategies = df_cleaned['strategy'].unique()

print(unique_strategies)

['EXAM:STRAT1' 'EXAM:STRAT2']


Writing a function that takes as input a strategy name and returns the grouped content of the strategy.

In [8]:
def group_by_strat(df_cleaned, strategy_name):
    df_cleaned_filtered = df_cleaned[df_cleaned['strategy'] == strategy_name]

    
    df_cleaned_grouped = df_cleaned_filtered.groupby(['ticker', 'description', 'trade_type']).agg({
        'notional': 'sum',
        'daily_pl': 'sum',
        'price_local': 'mean'
    }).reset_index()
    
    return df_cleaned_grouped

strategy_name = 'EXAM:STRAT2' #edit strategy here do not remove line from cell with function
grouped_data = group_by_strat(df_cleaned, strategy_name)

print("\nGrouped Data for Strategy '{}':".format(strategy_name))
display(grouped_data)


Grouped Data for Strategy 'EXAM:STRAT2':


,ticker,description,trade_type,notional,daily_pl,price_local
0,SFRH4 Comdty,USD SOFR 3M CME MAR24,RatesShortTermFuture,-57.0,0.0,94.655000
1,SFRH5 Comdty,USD SOFR 3M CME MAR25,RatesShortTermFuture,-29.0,-5876.0,95.300000
2,SFRH6 Comdty,USD SOFR 3M CME MAR26,RatesShortTermFuture,-19.0,-3500.0,95.865000
3,SFRM4 Comdty,USD SOFR 3M CME JUN24,RatesShortTermFuture,-30.0,-1486.0,94.715000
4,SFRM5 Comdty,USD SOFR 3M CME JUN25,RatesShortTermFuture,-30.0,-6689.0,95.500000
5,SFRU4 Comdty,USD SOFR 3M CME SEP24,RatesShortTermFuture,-30.0,-3007.0,94.875000
6,SFRU5 Comdty,USD SOFR 3M CME SEP25,RatesShortTermFuture,116.0,26210.0,95.670000
7,SFRZ4 Comdty,USD SOFR 3M CME DEC24,RatesShortTermFuture,-30.0,-4459.0,95.080000
8,SFRZ5 Comdty,USD SOFR 3M CME DEC25,RatesShortTermFuture,147.0,29451.0,95.785000
9,T 0 1/4 09/30/25,T 0 1/4 09/30/25,RatesBond,157452685.0,148687.0,93.552734


I am confused about the last line because I filtered out zero positions, I am going to check the original df to see if there happens to be an exact offsetting position in the strategy

In [9]:
print(df.loc[df['description'] == 'T 5 09/30/25'])

    Unnamed: 0  index     strategy trade_type    notional        ticker  \
6            6   2219  EXAM:STRAT2  RatesBond  86417500.0  T 5 09/30/25   
23          23   3229  EXAM:STRAT2  RatesBond -86417500.0  T 5 09/30/25   

     description  daily_pl  price_local  
6   T 5 09/30/25   70880.0    99.931641  
23  T 5 09/30/25  -70880.0    99.931641  


confirmed, there are two offsetting positons